In [6]:
!pip install opencv-python numpy tensorflow

In [7]:
!nvidia-smi

Sat Apr  1 21:46:57 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 511.69       Driver Version: 511.69       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:02:00.0 Off |                  N/A |
| N/A   60C    P0    N/A /  N/A |      0MiB /  2048MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 537125051926772312
xla_global_id: -1
]


In [9]:
from tensorflow.keras.models import load_model

model_path = 'CNNModel.h5'
model = load_model(model_path)

In [14]:
import cv2
import numpy as np
from keras.models import load_model

# Load the trained model
model = load_model("CNNModel.h5")

# Define the labels for the sign language alphabet
labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']

# Start capturing video from webcam
cap = cv2.VideoCapture(0)

# Define the region of interest for hand detection
roi_x, roi_y, roi_w, roi_h = 100, 100, 300, 300

# Define font and color for displaying the predicted label
font = cv2.FONT_HERSHEY_SIMPLEX
color = (0, 255, 0)
thickness = 2

# Define a variable to store the predicted label
predicted_label = ""


# Define a loop to capture frames and generate captions in real-time
while True:
    # Read a frame from the video stream
    ret, frame = cap.read()
    if not ret:
        break

    # Flip the frame horizontally for mirror effect
    frame = cv2.flip(frame, 1)

    # Extract the region of interest for hand detection
    roi = frame[roi_y:roi_y+roi_h, roi_x:roi_x+roi_w]

    # Preprocess the frame for feeding to the model
    roi = cv2.resize(roi, (32, 32))
    roi = np.array(roi, dtype=np.float32)
    roi = roi.reshape(1, 32, 32, 3)
    roi /= 255.0
    
#     # Write the predicted label to the output file
#     output_file.write(predicted_label + "\n")

    # Use the model to predict the label for the hand sign
    predictions = model.predict(roi)
    predicted_class = np.argmax(predictions)
    predicted_label = labels[predicted_class]

    # Display the predicted label on the frame
    cv2.putText(frame, predicted_label, (50, 50), font, 1, color, thickness, cv2.LINE_AA)

    # Display the frame
    cv2.imshow("Sign Language Captioning", frame)

    # Wait for user input to quit
    if cv2.waitKey(1) == ord('q'):
        break
        
# Save the captions to a file and remove repeated captions
captions = list(set(captions))
with open("captions.txt", "w") as f:
    for caption in captions:
        f.write(caption + "\n")

# Release the video capture and destroy all windows
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 29ms/step


NameError: name 'captions' is not defined